In [2]:
import os
import pandas as pd
import numpy as np



# Load dataset

In [3]:
path = '../data/nlp/20news-bydate-train/'
array = []
for dir1 in os.listdir(path):
    if os.path.isdir(os.path.join(path, dir1)):
        
        for file1 in os.listdir(os.path.join(path, dir1)):
            #print(os.path.join(path, dir1, file1))
            dct = {'From': '', 'Lines': '', 'Subject': '', 'Organization': ''}
            with open(os.path.join(path, dir1, file1), 'r', encoding='utf-8') as f:
                try:
                    lines = f.readlines()
                except:
                    print('Decoding error: {0}'.format(os.path.join(path, dir1, file1)))
            for i, line in enumerate(lines):
                splitted = line.strip().split(': ')
                if len(splitted) < 2:
                    break
                dct[splitted[0]] = splitted[1]

            message = '\n'.join(lines[i:])
            array.append((dct['From'], dct['Lines'], dct['Subject'], dct['Organization'], message, dir1))


Decoding error: ../data/nlp/20news-bydate-train/sci.electronics/53803
Decoding error: ../data/nlp/20news-bydate-train/sci.electronics/53883
Decoding error: ../data/nlp/20news-bydate-train/sci.electronics/53721
Decoding error: ../data/nlp/20news-bydate-train/sci.med/59239
Decoding error: ../data/nlp/20news-bydate-train/sci.med/59055
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/50467
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51591
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51892
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51592
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51916
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51941
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51865
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51904
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.m

In [4]:
df = pd.DataFrame(array, columns=['From', 'Lines', 'Subject', 'Organization', 'Message', 'Category'])

In [5]:
path = '../data/nlp/20news-bydate-test/'

array = []
for dir1 in os.listdir(path):
    if os.path.isdir(os.path.join(path, dir1)):
        
        for file1 in os.listdir(os.path.join(path, dir1)):
            #print(os.path.join(path, dir1, file1))
            dct = {'From': '', 'Lines': '', 'Subject': '', 'Organization': ''}
            with open(os.path.join(path, dir1, file1), 'r', encoding='utf-8') as f:
                try:
                    lines = f.readlines()
                except:
                    print('Decoding error: {0}'.format(os.path.join(path, dir1, file1)))
            for i, line in enumerate(lines):
                splitted = line.strip().split(': ')
                if len(splitted) < 2:
                    break
                dct[splitted[0]] = splitted[1]

            message = '\n'.join(lines[i:])
            array.append((dct['From'], dct['Lines'], dct['Subject'], dct['Organization'], message, dir1))

df_test = pd.DataFrame(array, columns=['From', 'Lines', 'Subject', 'Organization', 'Message', 'Category'])

Decoding error: ../data/nlp/20news-bydate-test/sci.electronics/54485
Decoding error: ../data/nlp/20news-bydate-test/sci.electronics/54071
Decoding error: ../data/nlp/20news-bydate-test/sci.electronics/54070
Decoding error: ../data/nlp/20news-bydate-test/sci.med/59535
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54042
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54000
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/53879
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54036
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54769
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54516
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52164
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52196
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52033
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52165
Decoding err

In [6]:
df_train = df
print(df_train.shape)
print(df_test.shape)

(11314, 6)
(7532, 6)


In [7]:
from sklearn.preprocessing import LabelEncoder
df_train['Target'] = LabelEncoder().fit_transform(df_train['Category'])
df_test['Target'] = LabelEncoder().fit_transform(df_test['Category'])

In [8]:
df_train['Re'] = (df_train['Subject'] == 'Re').map(lambda x: 1 if x else 0)
df_test['Re'] = (df_test['Subject'] == 'Re').map(lambda x: 1 if x else 0)
df_train['Subject'] = df_train['Subject'].map(lambda x: '' if x == 'Re' else x)
df_test['Subject'] = df_test['Subject'].map(lambda x: '' if x == 'Re' else x)

# Get features

In [9]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if \
              len([letter for letter in word if letter.isalpha()]) > 2 and '.' not in word]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


In [10]:
tfidf_vectorizer_message = TfidfVectorizer(max_df=0.5, max_features=200000,
                                 min_df=10, stop_words='english',
                                use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_message_train = tfidf_vectorizer_message.fit_transform(df_train['Message'])

In [11]:
tfidf_message_test = tfidf_vectorizer_message.transform(df_test['Message'])

In [12]:
tfidf_vectorizer_subject = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=2, stop_words='english',
                                use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_subject_train = tfidf_vectorizer_subject.fit_transform(df_train['Subject'])
tfidf_subject_test = tfidf_vectorizer_subject.transform(df_test['Subject'])

# Classify

## Train only with messages

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(tfidf_message_train, df_train['Target'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
print('Accuracy on the test set, using only messages: {0}'\
      .format(clf.score(tfidf_message_test, df_test['Target'])))

Accuracy on the test set, using only messages: 0.7700477960701009


## Train also with subjects

In [15]:
from scipy.sparse import hstack
train_matrix = hstack([tfidf_message_train, tfidf_subject_train])
test_matrix = hstack([tfidf_message_test, tfidf_subject_test])

clf_subject = MultinomialNB()
clf_subject.fit(train_matrix, df_train['Target'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
print('Accuracy on the test set, using messages and subjects: {0}'\
      .format(clf_subject.score(test_matrix, df_test['Target'])))

Accuracy on the test set, using messages and subjects: 0.7756240042485396


# Cluster

In [17]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=20, random_state=0)
km.fit_predict(train_matrix)

array([ 4,  4, 11, ..., 12,  9,  9], dtype=int32)

In [18]:
from sklearn import metrics

labels = df_train['Target']
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(train_matrix, km.labels_, sample_size=1000))

Homogeneity: 0.360
Completeness: 0.471
V-measure: 0.408
Adjusted Rand-Index: 0.123
Silhouette Coefficient: -0.043


In [19]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms1 = tfidf_vectorizer_message.get_feature_names()
terms2 = tfidf_vectorizer_subject.get_feature_names()
terms = terms1 + terms2
for i in range(20):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: question question thank ani doe answer use just know x-termin
Cluster 1: geb gordon bank shame surrend soon shame surrend surrend soon geb shame surrend geb shame chastiti intellect geb intellect geb intellect geb shame
Cluster 2: armenian turkish armenian armenia turk muslim turkey azeri genocid azerbaijan
Cluster 3: peopl govern moral articl right law think just say state
Cluster 4: articl use just like ani know think post time doe
Cluster 5: game team player play win year season hockey score fan
Cluster 6: problem problem window error use run thank screen tri ani
Cluster 7: drive forsal scsi drive ide disk hard drive hard floppi control
Cluster 8: want thank look pleas email e-mail info want want game anyon
Cluster 9: file file window faq program format use imag directori ftp
Cluster 10: thank use sale anyon ani pleas doe know need card
Cluster 11: car bike ride articl dod just like ani engin look
Cluster 12: help need need help help help need thank ani pleas use appreci


In [20]:
len(terms)

20110

In [21]:
order_centroids.shape

(20, 20110)

## Only with messages

In [22]:
km = KMeans(n_clusters=20, random_state=0)
km.fit_predict(tfidf_message_train)

array([ 6,  0,  0, ..., 11,  7,  7], dtype=int32)

In [23]:

labels = df_train['Target']
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(tfidf_message_train, km.labels_, sample_size=1000))

Homogeneity: 0.401
Completeness: 0.479
V-measure: 0.437
Adjusted Rand-Index: 0.144
Silhouette Coefficient: 0.011


In [24]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer_message.get_feature_names()
for i in range(20):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: bike ride dod motorcycl dog helmet rider articl max bmw
Cluster 1: armenian turkish turk armenia turkey greek azeri genocid peopl azerbaijan
Cluster 2: moral object islam keith livesey muslim valu frank jon articl
Cluster 3: drive scsi disk ide hard drive hard floppi control problem boot
Cluster 4: fbi koresh robi gas cdt batf compound children tear gas tear
Cluster 5: window driver dos card use version run problem program video
Cluster 6: articl just like use think ani know time post say
Cluster 7: file directori window program format use ftp imag ani convert
Cluster 8: doe anyon anyon know doe anyon know anyon doe know thank ani anyon know ani program
Cluster 9: geb gordon bank chastiti intellect geb intellect geb intellect geb shame shame surrend soon shame surrend geb shame surrend surrend soon geb shame
Cluster 10: gun peopl govern law right weapon state think crime make
Cluster 11: thank pleas ani thank advanc advanc e-mail appreci email anyon mail
Cluster 12: car engi